## Part 3: Reading Vector Data with Fiona

https://fiona.readthedocs.io/en/stable/manual.html

https://github.com/Toblerity/Fiona

**Execute the next 2 lines of code (download) while I'm explaining**

Fiona streams simple feature data to and from GIS formats like GeoPackage and Shapefile.

Fiona can read and write real-world data using multi-layered GIS formats, zipped and in-memory virtual file systems, from files on your hard drive or in cloud storage. This project includes Python modules and a command line interface (CLI).

Fiona depends on GDAL but is different from GDAL's own bindings. Fiona is designed to be highly productive and to make it easy to write code which is easy to read.

## Vectors ? Types of Geospatial Data

The Open Geospatial Consortium (OGC) Simple Feature Access Standard defines a common set of geometries for representing two-dimensional geospatial features in vector data. These geometries are widely used in GIS software and data formats to describe the spatial characteristics of features like points, lines, and polygons. The standard specifies the following geometry types:
    
![Geometry Types](artwork/geomtype.png)

    
### Point

A single location in space, defined by a pair of coordinates (x, y). Points can represent features like trees, streetlights, or the centroids of more complex shapes.

### LineString

A sequence of connected points, forming a line. LineStrings are used to represent linear features like roads, rivers, and paths. LineStrings can be open or closed; a closed LineString forms a linear ring.

### Polygon

A closed, two-dimensional shape defined by one or more linear rings. The first ring represents the outer boundary of the polygon, while additional rings, if present, represent holes or interior boundaries. Polygons are used to represent areas like buildings, lakes, or administrative boundaries.

### MultiPoint

A collection of one or more Point geometries, which can be used to represent a group of discrete features that share the same attributes.

### MultiLineString

A collection of one or more LineString geometries, which can represent a group of linear features that share the same attributes. Examples include a network of roads or a group of disconnected rivers.

### MultiPolygon

A collection of one or more Polygon geometries, which can represent a group of area features that share the same attributes. Examples include a group of islands or a set of non-contiguous administrative regions.

### GeometryCollection

A collection of any combination of the geometry types listed above. This allows for more complex representations of features that have multiple, distinct geometries with different types.


Those types are covered in the **shapely** Module.


## Popular Vector File Formats

Vector formats are used to represent geospatial data as points, lines, and polygons, which are composed of coordinates defining their shapes and positions. Vector data is distinct from raster data, which represents geospatial information as a grid of pixels or cells, each having a specific value. Vector formats are particularly suited for representing discrete features like roads, buildings, rivers, or administrative boundaries.

Here is an **incomplete** list of some popular vector data formats:

### Shapefile (.shp)

Shapefile (.shp): Developed by Esri, this is a widely used and supported format for storing vector data. It consists of multiple files, including geometry, attribute data, and projection information.

Do not use Shapefiles. http://switchfromshapefile.org/

### GeoJSON (.geojson)

A lightweight, text-based format that uses the JSON (JavaScript Object Notation) standard to represent geographic features and their properties. It is especially popular for web-based mapping and geospatial applications.

### KML (.kml) and KMZ (.kmz)

Keyhole Markup Language (KML) is an XML-based format originally developed for Google Earth. It is used to display geographic data in Earth browsers and other compatible applications. KMZ is a compressed version of KML.

### MicroStation DGN (.dgn)

A CAD-based vector format developed by Bentley Systems for their MicroStation software.

### Esri File Geodatabase (.gdb) 

A proprietary data format developed by Esri for storing and managing geospatial data, including vector features, raster data, and attribute tables. File geodatabases offer efficient storage, spatial indexing, and compression options, making them suitable for large datasets and complex applications.

### OGC Geopackage (.gpkg)

The GeoPackage format is an open, standards-based, platform-independent, and portable data format for geospatial information. Developed by the Open Geospatial Consortium (OGC), it is designed to facilitate the storage, sharing, and management of geospatial data, including vector features, tile matrix sets of imagery and raster maps at various scales, and even simple nontopological attributes.

GeoPackage files use the SQLite database format, with the file extension ".gpkg". SQLite is a widely used, lightweight, and self-contained database engine, which makes GeoPackage files easily portable and accessible across different devices, platforms, and programming languages.

( More info: https://www.geopackage.org/ )


### SpatiaLite (.sqlite)

An extension to the SQLite database format that adds support for geospatial data types and operations. SpatiaLite is similar to the GeoPackage format but follows a different specification.





## Download Sample Vector Data

Let's download some data we need later. We download data in the **GeoPackage** format.

**Natural Earth** is a public domain map dataset available at 1:10m, 1:50m, and 1:110 million scales. Featuring tightly integrated vector and raster data, with Natural Earth you can make a variety of visually pleasing, well-crafted maps with cartography or GIS software. More info: https://www.naturalearthdata.com/

We download vector data now

In [ ]:
from download import download, unzip

In [ ]:
url = "http://naciscdn.org/naturalearth/packages/natural_earth_vector.gpkg.zip"
#url = "https://www.geopython.xyz/geodata/naturalearth/natural_earth_vector.gpkg.zip" # backup-url

download(url, "geodata/ne.gpkg.zip")
unzip("geodata/ne.gpkg.zip", "geodata")

## Reading Vector File

In [ ]:
import fiona

List supported vector file formats:

In [ ]:
fiona.supported_drivers

In [ ]:
if 'GPKG' in fiona.supported_drivers:
    print("Yay!! GeoPackageis supported...")

### Layers in GeoPackage 

In a GeoPackage, layers, also known as tables, are used to organize and store different types of geospatial data within the same SQLite database file. Each layer is a container for a specific type of geospatial data and its associated attributes. 


Let's list all available layers of our Geopackage file:

In [ ]:
filename = "geodata/packages/natural_earth_vector.gpkg"  # this is the extracted GeoPackage file

In [ ]:
layers = fiona.listlayers(filename)
layers

### Reading a Layer

Let's use the `ne_10m_airports` layer. It cointains some airports around the workd. 

10m stands for the resolution 1:10_000_000 resolution. Some data is also available in 50m and 110m. (1:50_000_000 and 1:110_000_000)

Note that many other vector file formats don't have the concept of layers (Shapefile, GeoJSON, ...). In that case you can't list layers and you would just omit the layer parameter when opening the file.

Open the dataset:

In [ ]:
c = fiona.open(filename, 'r', layer='ne_10m_airports')

#### Get the schema

In [ ]:
c.schema

#### Attributes / Properties

In [ ]:
c.schema["properties"]

#### Get the Geospatial Data Type

In [ ]:
c.schema["geometry"]

#### Get the first entry:

In [ ]:
airport = next(iter(c))

In [ ]:
airport['properties']['name']

In [ ]:
airport['geometry']['type']

In [ ]:
airport['geometry']['coordinates']

In [ ]:
airport["properties"]["name_hi"]

In [ ]:
c.crs

In [ ]:
c.close()

#### Retrieving all data

There are basically two ways:

* convert to a list: Load everything into memory: (if dataset isn't too large...)

      alldata = list(c)
    
* iterate through all data: (one by one):

      for element in c:
           ...

In [ ]:
cnt = 0
with fiona.open(filename, 'r', layer='ne_10m_airports') as c:
    for airport in c:
        if cnt < 15:
             print(dict(airport['properties'])['name'])
        cnt +=1

In [ ]:
with fiona.open(filename, 'r', layer='ne_10m_airports') as c:
    for airport in c:
        if airport['properties']['iata_code'] == "TXL":
            #print(airport['properties']['name'])
            #print(airport['geometry']['coordinates'])
            #print(airport['properties']['wikipedia'])
            print(dict(airport['properties']))
            print(dict(airport['geometry']))

In [ ]:
with fiona.open(filename, 'r', layer='ne_10m_airports') as c:
    print(c.crs)

In [ ]:
import fiona

c = fiona.open(filename, 'r', layer='ne_10m_admin_0_countries')

c.schema

In [ ]:
country = next(iter(c))

print(country['properties']['NAME'])
print(country['properties']['NAME_ZH'])
print(country['properties']['CONTINENT'])
print(country['properties']['POP_EST'])
print(country['properties']['POP_YEAR'])

In [ ]:
with fiona.open(filename, 'r', layer='ne_10m_admin_0_countries') as c:
    for country in c:
        if country['properties']['NAME'] == "Germany":
            print(country['properties']['POP_EST'])
            print(country['properties']['POP_YEAR'])   
            print(country['geometry']['type'])
            print(country['geometry']['coordinates'])               

In [ ]:
 print(dict(airport['properties']))